In [ ]:
import pandas as pd
import openpyxl
from pptx import Presentation
from tqdm import tqdm
import math
import os
from pptx.util import Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.oxml.xmlchemy import OxmlElement

class LRDM:
    def __init__(self, **args):
        self.input = "./input/"
        self.demographics_file = self.input + args['demographics']
        self.sample_file = self.input + "/templates/" + args['sample']
        self.analyst_file = self.input + args['analyst']
        self.GM_levels_file = self.input + args['gm_levels']
        self.leaders_file = self.input + args['leaders']

    def readAllFiles(self):
        self.demographics_pd = pd.read_excel(self.demographics_file, engine="openpyxl")

        self.analyst_leader_pd = pd.read_excel(self.analyst_file, engine="openpyxl", sheet_name="Leader")
        self.analyst_GM_pd = pd.read_excel(self.analyst_file, engine="openpyxl", sheet_name="GM")
        self.analyst_Site_pd = pd.read_excel(self.analyst_file, engine="openpyxl", sheet_name="Site Leader")

        self.GM_levels_pd = pd.read_excel(self.GM_levels_file, engine="openpyxl")

        self.leaders_pd = pd.read_excel(self.leaders_file, engine="openpyxl", sheet_name="Leader")
        self.GMs_pd = pd.read_excel(self.leaders_file, engine="openpyxl", sheet_name="GM")
        self.Site_leaders_pd = pd.read_excel(self.leaders_file, engine="openpyxl", sheet_name="Site Leader")

    def setLeader(self, id, GM=False, site_lead=False):
        self.leader_id = id
        self.GM = GM
        self.site_lead = site_lead

    def prepareContent(self):
        no_direct = False
        
        _temp = "Supervisor Level {} ID"

        _leader_entry = self.demographics_pd[self.demographics_pd.loc[:, "Worker ID"] == self.leader_id]
        _last_name = _leader_entry['Worker Last Name'].values[0]
        _full_name = _leader_entry['Worker Name'].values[0]
        self.first_content = [_full_name, _last_name + " Organization"]
        self.third_content = []
        self.seventh_dict = {}

        ## make demographics data in order to get your org.
        demographics_data = self.demographics_pd[self.demographics_pd["Invitee Flag"] == 1].reset_index(drop=True)
        if self.GM:
            demographics_data = demographics_data[demographics_data[self.GM] == 1].reset_index(drop=True)
        elif self.site_lead:
            demographics_data = demographics_data[demographics_data[self.site_lead] == 1].reset_index(drop=True)
        your_org = demographics_data
        temp_org = your_org[~(your_org.loc[:, "Worker ID"] == self.leader_id)].reset_index(drop=True)

        def get_leader_level(_leader_entry, _leader_id):
            for index, level in enumerate(_leader_entry.loc[:, _temp.format(2) : _temp.format(10)]):
                if (_leader_entry[level] == _leader_id).tolist()[0]:
                    _leader_level = index + 2
                    break
            return _leader_level
        
        _leader_level = get_leader_level(_leader_entry, self.leader_id)

        _supervisor_level = _leader_level - 1
        _direct_level = _leader_level + 1

        temp_org = temp_org[temp_org.loc[:, _temp.format(_leader_level)] == self.leader_id].reset_index(drop=True)
        _dict = temp_org.groupby(_temp.format(_direct_level)).groups
        direct_items = _dict.keys()

        use_affiliate = False
        affiliate_second = False

        if self.GM:
            _temp_level = "GM Level {} ID"

            self.first_content = [self.GM[:-4], self.GM[:-4]]
            _analyst_entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == self.leader_id]
            _gilead_entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == 999999]

            self.seventh_dict.update({"Gilead Overall": [_gilead_entry.iloc[:, 21: 31].values.tolist(), 0]})
            self.third_content.append(_analyst_entry.iloc[:, 2: 6].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 7: 12].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 12: 15].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 15: 17].values.tolist())

            try:
                if len(self.GM_levels_pd[self.GM_levels_pd["GM Level 2 ID"] == self.leader_id].index) > 1:
                    use_affiliate = True
            except:
                pass
            try:
                if len(self.GM_levels_pd[self.GM_levels_pd["GM Level 3 ID"] == self.leader_id].index) > 1:
                    use_affiliate = True
                    affiliate_second = True
            except:
                pass

            if use_affiliate:
                self.seventh_dict.update({self.GM: [_analyst_entry.iloc[:, 21: 31].values.tolist(), 1]})
                temp_id = "GM Level {} ID"
                temp_name = "GM Level {} Org"
                key_list = []
                def get_org_id_by_name(name):
                    return self.GM_levels_pd[self.GM_levels_pd["GM Org"] == name]["GM ID"].values[0]
                
                if not affiliate_second:
                    _3_levels = self.GM_levels_pd[self.GM_levels_pd[temp_id.format(2)] == self.leader_id]
                    _3_list = _3_levels[~(_3_levels["GM ID"] == self.leader_id)][temp_name.format(3)].drop_duplicates().values
                    _3_list = sorted(_3_list)
                    _3_last = []
                    _3_complete = []
                    for item in _3_list[:]:
                        if item.startswith("Kite"):
                            _3_last.append(item)
                        else:
                            _3_complete.append(item)
                    for item in _3_last:
                        _3_complete.append(item)

                    for _3_name in _3_complete:
                        key_list.append(" " + _3_name)
                        _3_id = get_org_id_by_name(_3_name)
                        _4_levels = self.GM_levels_pd[self.GM_levels_pd[temp_id.format(3)] == _3_id]
                        _4_list = _4_levels[~(_4_levels["GM ID"] == _3_id)][temp_name.format(4)].values
                        _4_list = sorted(_4_list)
                        _4_last = []
                        _4_complete = []
                        for item in _4_list:
                            if item.startswith("Kite"):
                                _4_last.append(item)
                            else:
                                _4_complete.append(item)
                        for item in _4_last:
                            _4_complete.append(item)

                        for _4_name in _4_complete:
                            key_list.append("  " + _4_name)
                    
                    real_list = []
                    for key in key_list:
                        real_key = key.strip()
                        try:
                            real_list.append([get_org_id_by_name(real_key), key])
                        except:
                            pass
                    
                    for real_item in real_list:
                        _entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == real_item[0]]
                        self.seventh_dict.update({real_item[1]: [_entry.iloc[:, 21: 31].values.tolist(), 0]})

                    ## 7-4 note
                    self.template_file = self.sample_file + " 4.pptx"
                
                else:
                    _4_levels = self.GM_levels_pd[self.GM_levels_pd[temp_id.format(3)] == self.leader_id]
                    _4_list = _4_levels[~(_4_levels["GM ID"] == self.leader_id)][temp_name.format(4)].values
                    _4_list = sorted(_4_list)
                    _4_last = []
                    _4_complete = []
                    for item in _4_list:
                        if item.startswith("Kite"):
                            _4_last.append(item)
                        else:
                            _4_complete.append(item)
                    for item in _4_last:
                        _4_complete.append(item)
                    
                    for _4_name in _4_complete:
                        key_list.append(" " + _4_name)

                    real_list = []
                    for key in key_list:
                        real_key = key.strip()
                        try:
                            real_list.append([get_org_id_by_name(real_key), key])
                        except:
                            pass
                    
                    for real_item in real_list:
                        _entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == real_item[0]]
                        self.seventh_dict.update({real_item[1]: [_entry.iloc[:, 21: 31].values.tolist(), 0]})

                    ## 7-5 note
                    self.template_file = self.sample_file + " 5.pptx"


            _single = self.GM_levels_pd[self.GM_levels_pd["GM ID"] == self.leader_id]
            _2_id = _single["GM Level 2 ID"].values[0]
            _2_name = self.GM_levels_pd[self.GM_levels_pd["GM ID"] == _2_id]["GM Org"].values[0]
            _2_entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == _2_id]
            if _single["GM Level 3 ID"].values[0] != self.leader_id:
                self.seventh_dict.update({_2_name: [_2_entry.iloc[:, 21: 31].values.tolist(), 0]})
                _3_id = _single["GM Level 3 ID"].values[0]
                _3_name = self.GM_levels_pd[self.GM_levels_pd["GM ID"] == _3_id]["GM Org"].values[0]
                _3_entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == _3_id]
                self.seventh_dict.update({" " + _3_name: [_3_entry.iloc[:, 21: 31].values.tolist(), 0]})

                _4_heap = self.GM_levels_pd[self.GM_levels_pd["GM Level 3 ID"] == _3_id]
                _4_heap = _4_heap[~(_4_heap["GM ID"] == _3_id)]
                _4_heap = _4_heap.sort_values(by=["GM Org"])

                for index, row in _4_heap.iterrows():
                    _name = row["GM Org"]
                    _entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == row["GM ID"]]
                    if row["GM ID"] == self.leader_id:
                        self.seventh_dict.update({"  " + _name: [_entry.iloc[:, 21: 31].values.tolist(), 1]})
                    else:
                        self.seventh_dict.update({"  " + _name: [_entry.iloc[:, 21: 31].values.tolist(), 0]})
                
                ## 7-6 note
                self.template_file = self.sample_file + " 6.pptx"
                        
            elif _single["GM Level 2 ID"].values[0] != self.leader_id and len(self.GM_levels_pd[self.GM_levels_pd["GM Level 3 ID"] == self.leader_id].index) == 1:
                self.seventh_dict.update({_2_name: [_2_entry.iloc[:, 21: 31].values.tolist(), 0]})
                _3_over = self.GM_levels_pd.drop_duplicates(subset=["GM Level 3 ID"])

                _3_heap = _3_over[_3_over["GM Level 2 ID"] == _2_id]
                _3_heap = _3_heap[~(_3_heap["GM ID"] == _2_id)]
                _3_heap = _3_heap.sort_values(by=["GM Org"])

                for index, row in _3_heap.iterrows():
                    _name = row["GM Org"]
                    _entry = self.analyst_GM_pd[self.analyst_GM_pd["GM ID"] == row["GM ID"]]
                    if row["GM ID"] == self.leader_id:
                        self.seventh_dict.update({" " + _name: [_entry.iloc[:, 21: 31].values.tolist(), 1]})
                    else:
                        self.seventh_dict.update({" " + _name: [_entry.iloc[:, 21: 31].values.tolist(), 0]})
                    
                ## 7-6 note
                self.template_file = self.sample_file + " 6.pptx"

            elif len(self.GM_levels_pd[self.GM_levels_pd["GM Level 2 ID"] == self.leader_id].index) == 1:
                self.seventh_dict.update({self.GM: [_analyst_entry.iloc[:, 21: 31].values.tolist(), 1]})

                ## 7-7 note
                self.template_file = self.sample_file + " 7.pptx"
    
        elif self.site_lead:

            self.first_content = [self.site_lead.replace(" / ", " "), self.site_lead.replace(" / ", " ")]
            _analyst_entry = self.analyst_Site_pd[self.analyst_Site_pd["Site Leader ID"] == self.leader_id]

            _gilead_entry = self.analyst_Site_pd[self.analyst_Site_pd["Site Leader ID"] == 999999]
            _analyst_directs = self.analyst_leader_pd[self.analyst_leader_pd["Leader ID"].isin(direct_items)]
            if _analyst_directs.empty:
                no_direct = True
            _analyst_directs = _analyst_directs.iloc[:, 27: 37].sum(axis=0, skipna=True)

            self.seventh_dict.update({"Gilead Overall": [_gilead_entry.iloc[:, 21: 31].values.tolist(), 0]})
            self.third_content.append(_analyst_entry.iloc[:, 2: 6].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 7: 12].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 12: 15].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 15: 17].values.tolist())

            _leaders = self.analyst_leader_pd[~(self.analyst_leader_pd["Leader ID"].isin([999999, self.leader_id]))].reset_index(drop=True)
            _leaders = _leaders[_leaders["Leader ID"].notna()]
            _ids = _leaders["Leader ID"].values.tolist()

            __ids = _leader_entry.loc[:, _temp.format(2) : _temp.format(_leader_level - 1)].values.tolist() 
            _ids = list(filter(lambda item: True if item in _ids else False , __ids[0]))

            for _id in _ids:
                _entry = self.demographics_pd[self.demographics_pd.loc[:, "Worker ID"] == _id]
                _a_entry =  self.analyst_leader_pd[self.analyst_leader_pd["Leader ID"] == _id]
                _last_name = _entry['Worker Last Name'].values[0]
                self.seventh_dict.update({"{} Org".format(_last_name): [_a_entry.iloc[:, 27: 37].values.tolist(), 0]})

            self.seventh_dict.update({"Your Org": [_analyst_entry.iloc[:, 21: 31].values.tolist(), 1]})
            if no_direct:
                self.seventh_dict.update({"Your Directs’ Orgs": [[_analyst_directs.tolist()], 0]})
                ## 7-3 note
                self.template_file = self.sample_file + " 3.pptx"
            else :
                ## 7-2 note
                self.template_file = self.sample_file + " 2.pptx"

        else:
            _analyst_entry = self.analyst_leader_pd[self.analyst_leader_pd["Leader ID"] == self.leader_id]
            _gilead_entry = self.analyst_leader_pd[self.analyst_leader_pd["Leader ID"] == 999999]
            _analyst_directs = self.analyst_leader_pd[self.analyst_leader_pd["Leader ID"].isin(direct_items)]

            if _analyst_directs.empty:
                no_direct = True
            _analyst_directs = _analyst_directs.iloc[:, 27: 37].sum(axis=0, skipna=True)

            self.seventh_dict.update({"Gilead Overall": [_gilead_entry.iloc[:, 27: 37].values.tolist(), 0]})
            self.third_content.append(_analyst_entry.iloc[:, 3: 7].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 8: 13].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 13: 16].values.tolist())
            self.third_content.append(_analyst_entry.iloc[:, 16: 18].values.tolist())

            if _leader_level != 2:
                _leaders = self.analyst_leader_pd[~(self.analyst_leader_pd["Leader ID"].isin([999999, self.leader_id]))].reset_index(drop=True)
                _leaders = _leaders[_leaders["Leader ID"].notna()]
                _ids = _leaders["Leader ID"].values.tolist()

                __ids = _leader_entry.loc[:, _temp.format(2) : _temp.format(_leader_level - 1)].values.tolist() 
                _ids = list(filter(lambda item: True if item in _ids else False , __ids[0]))

                for _id in _ids:
                    _entry = self.demographics_pd[self.demographics_pd.loc[:, "Worker ID"] == _id]
                    _a_entry =  self.analyst_leader_pd[self.analyst_leader_pd["Leader ID"] == _id]
                    _last_name = _entry['Worker Last Name'].values[0]
                    self.seventh_dict.update({"{} Org".format(_last_name): [_a_entry.iloc[:, 27: 37].values.tolist(), 0]})

                ## 7-2 note
                self.template_file = self.sample_file + " 2.pptx"

            ## 7-1 note
            if _leader_level == 2:
                self.template_file = self.sample_file + " 1.pptx"

            self.seventh_dict.update({"Your Org": [_analyst_entry.iloc[:, 27: 37].values.tolist(), 1]})
            if no_direct:
                self.seventh_dict.update({"Your Directs’ Orgs": [[_analyst_directs.tolist()], 0]})

                ## 7-3 note
                self.template_file = self.sample_file + " 3.pptx"
        self.prs = Presentation(self.template_file)


    def makePresentation(self):

        def sub_element(parent, tagname, **kwargs):
                element = OxmlElement(tagname)
                element.attrib.update(kwargs)
                parent.append(element)
                return element

        def set_cell_border(cell, side):
            tc = cell._tc
            tcPr = tc.get_or_add_tcPr()
            border = sub_element(tcPr, f'a:ln{side}', w='25000', algn='ctr', cmpd='sng', cap='flat')
            border_fill = sub_element(border, 'a:solidFill')
            border_scheme = sub_element(border_fill, 'a:schemeClr', val='bg2')
            border_lumMod = sub_element(border_scheme, 'a:lumMod', val='50000')
            border_dash = sub_element(border, 'a:prstDash', val='solid')
            border_round = sub_element(border, 'a:round')
            border_headEnd = sub_element(border, 'a:headEnd', type='none', w='med', len='med')
            border_tailEnd = sub_element(border, 'a:tailEnd', type='none', w='med', len='med')

        for index, slide in enumerate(self.prs.slides):
            if index == 0:
                shape = slide.shapes[1]
                paragraph = shape.text_frame.paragraphs[0]
                paragraph.text = self.first_content[1]
                # for run in paragraph.runs:
                #     run.text = self.first_content[1]
            elif index == 2:
                for sub_index, shape in enumerate(slide.shapes):
                    if sub_index == 2:

                        ## fill the first bullet list.
                        _third_content = [item for item in self.third_content[0][0] if type(item) == type('')]
                        if len(_third_content) > 3:
                            for _ in range(len(_third_content) - 3):
                                shape.table.cell(1, 1).text_frame.add_paragraph()

                        _cell = shape.table.cell(1, 1)
                        for end_index, paragraph in enumerate(_cell.text_frame.paragraphs):
                            try:
                                # paragraph.text = _third_content[end_index]
                                for run in paragraph.runs:
                                    run.text = _third_content[end_index]
                                    run.font.name = "Trebuchet"
                                    run.font.size = Pt(16)
                            except:
                                for run in paragraph.runs:
                                    run.text = ''

                        ## fill the second bullet list.
                        _third_content = [item for item in self.third_content[1][0] if type(item) == type('')]
                        if len(_third_content) > 2:
                            for _ in range(len(_third_content) - 2):
                                shape.table.cell(1, 2).text_frame.add_paragraph()

                        _cell = shape.table.cell(1, 2)
                        for end_index, paragraph in enumerate(_cell.text_frame.paragraphs):
                            try:
                                # paragraph.text = _third_content[end_index]
                                for run in paragraph.runs:
                                    run.text = _third_content[end_index]
                                    run.font.name = "Trebuchet"
                                    run.font.size = Pt(16)
                            except:
                                for run in paragraph.runs:
                                    run.text = ''

                        ## fill the third bullet list.
                        _third_content = [item for item in self.third_content[2][0] if type(item) == type('')]
                        if len(_third_content) > 2:
                            for _ in range(len(_third_content) - 2):
                                shape.table.cell(3, 1).text_frame.add_paragraph()

                        _cell = shape.table.cell(3, 1)
                        for end_index, paragraph in enumerate(_cell.text_frame.paragraphs):
                            try:
                                # paragraph.text = _third_content[end_index]
                                for run in paragraph.runs:
                                    run.text = _third_content[end_index]
                                    run.font.name = "Trebuchet"
                                    run.font.size = Pt(16)
                            except:
                                if end_index != 3:
                                    for run in paragraph.runs:
                                        run.text = ''
                                else:
                                    if len(_third_content) == 1:
                                        for run in paragraph.runs:
                                            run.text = ''
                        
                        if len(_third_content) <= 1:
                            for paragraph in shape.table.cell(2, 1).text_frame.paragraphs:
                                for run in paragraph.runs:
                                    run.text = run.text[:-1]
                                    run.font.name = "Trebuchet MS"
                                    run.font.size = Pt(24)
                            

                        ## fill the fourth bullet list.
                        _third_content = [item for item in self.third_content[3][0] if type(item) == type('')]
                        if len(_third_content) > 2:
                            for _ in range(len(_third_content) - 2):
                                shape.table.cell(3, 2).text_frame.add_paragraph()

                        _cell = shape.table.cell(3, 2)
                        for end_index, paragraph in enumerate(_cell.text_frame.paragraphs):
                            try:
                                # paragraph.text = _third_content[end_index]
                                for run in paragraph.runs:
                                    run.text = _third_content[end_index]
                                    run.font.name = "Trebuchet"
                                    run.font.size = Pt(16)
                            except:
                                for run in paragraph.runs:
                                    run.text = ''
                        
                        if len(_third_content) <= 1:
                            for paragraph in shape.table.cell(2, 2).text_frame.paragraphs:
                                for run in paragraph.runs:
                                    run.text = run.text[:-1]
                                    run.font.name = "Trebuchet MS"
                                    run.font.size = Pt(24)
            # elif index == 3:
            #     for index, shape in enumerate(slide.shapes):
            #         try:
            #             shape.fill.solid()
            #             shape.fill.fore_color.rgb = RGBColor(0, 255, 0)
            #             # print(index, shape)
            #         except:
            #             pass
            elif index == 6:
                for index, shape in enumerate(slide.shapes):
                    if shape.has_table:                        
                        ## pre process for customizing a table.
                        _len = len(shape.table.rows)
                        if int(self.template_file[-6]) == 4:
                            for _ in range(_len - 2):
                                shape.table.rows.remove(shape.table.rows[1])
                        else:
                            for _ in range(_len - 2):
                                shape.table.rows.remove(shape.table.rows[2])

                        [shape.table.rows.add() for _ in range(len(self.seventh_dict) - 1)]

                        color = [RGBColor(204, 205, 210), RGBColor(231, 232, 234)]
                        
                        for row_index, key in enumerate(self.seventh_dict):
                            for run in shape.table.cell(row_index + 1, 0).text_frame.paragraphs[0].runs:
                                if self.seventh_dict[key][1] == 1:
                                    set_cell_border(shape.table.cell(row_index + 1, 0), 'L')
                                    set_cell_border(shape.table.cell(row_index + 1, 0), 'T')
                                    set_cell_border(shape.table.cell(row_index + 1, 0), 'B')
                            
                            for col_index, value in enumerate(self.seventh_dict[key][0][0]):
                                cell = shape.table.cell(row_index + 1, col_index + 1)
                                if self.seventh_dict[key][1] == 1:
                                    if col_index == len(self.seventh_dict[key][0][0]) - 1:
                                        set_cell_border(cell, 'R')
                                    set_cell_border(cell, 'T')
                                    set_cell_border(cell, 'B')

                        for row_index in range(1, len(self.seventh_dict) + 1):
                            for col_index in range(11):
                                cell = shape.table.cell(row_index, col_index)
                                cell.fill.solid()
                                cell.fill.fore_color.rgb = color[(row_index + 1) % 2]

                        for row_index, key in enumerate(self.seventh_dict):
                            for run in shape.table.cell(row_index + 1, 0).text_frame.paragraphs[0].runs:
                                run.text = key
                                run.font.size = Pt(9)
                                if self.seventh_dict[key][1] == 1:
                                    run.font.color.rgb = RGBColor(60, 88, 127)
                                    run.font.size = Pt(12)

                            for col_index, value in enumerate(self.seventh_dict[key][0][0]):
                                cell = shape.table.cell(row_index + 1, col_index + 1)
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                try:
                                    if int(value) >= 1:
                                        cell.fill.fore_color.rgb = RGBColor(247, 143, 169)
                                except:
                                    pass

                                run = cell.text_frame.paragraphs[0].add_run()
                                if key == "Your Directs’ Orgs":
                                    try:
                                        if int(value) >= 1:
                                            run.text = str(int(value))
                                    except:
                                        pass
                                run.font.size = Pt(9)

    def writeOutput(self):
        _output_path = './output/' + self.first_content[0] + '/'
        _filename = "2021-04 Global Employee Survey - {} - Leader Results Deck.pptx"
        _filename = _filename.format(self.first_content[1])
        _path = _output_path + _filename

        if os.path.exists(_path):
            os.remove(_path)

        ## make a folder to involve the output file.
        os.makedirs(_output_path, exist_ok=True)

        self.prs.save(_path)

if __name__ == "__main__":
    init_data = {
        'demographics': "2020 Demographics File Sample 2021-02-17.xlsx",
        'sample': "Leader Results Deck Master Template",
        'analyst': "Analyst Insights Database Sample 2021-02-25.xlsx",
        'gm_levels': "GM Levels 2021-02-17.xlsx",
        'leaders': "List of Leader GM SiteLeader 2021-02-23.xlsx",
    }

    lrdm = LRDM(**init_data)

    lrdm.readAllFiles()

    total_ids = len(lrdm.leaders_pd.index) + len(lrdm.GMs_pd.index) + len(lrdm.Site_leaders_pd.index)
    for index in tqdm(range(total_ids), desc="LRDM process"):
        if index < len(lrdm.leaders_pd.index):
            if lrdm.leaders_pd.iloc[index, :].values[0] == 999999:
                continue
            lrdm.setLeader(lrdm.leaders_pd.iloc[index, :].values[0])
        elif index < len(lrdm.leaders_pd.index) + len(lrdm.GMs_pd.index):
            row = lrdm.GMs_pd.iloc[index - len(lrdm.leaders_pd.index), :]
            lrdm.setLeader(row["GM ID"], row["GM Org"])
        else:
            row = lrdm.Site_leaders_pd.iloc[index - len(lrdm.leaders_pd.index) - len(lrdm.GMs_pd.index), :]
            lrdm.setLeader(row["Site Leader ID"], False, row["Site Name"])

        lrdm.prepareContent()

        lrdm.makePresentation()

        lrdm.writeOutput()
    print("done")